# Combining Two Files

In [264]:
from load_trajectories import get_timestamps, load_trajectories, get_unique_SDs
# Using the NPL 2024a kernel
import os
import os.path

#for calculating the air temperature
from metpy.units import units
from metpy.calc import temperature_from_potential_temperature

#time
from datetime import timedelta

import dask
import dask.dataframe as dd
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import pyplot as plt

# Environmental Data File

In [265]:
dirpath = "/glade/derecho/scratch/klamb/superdroplets/outsdm_iceball_nowind_rhod_dist_min200_sgs_1024_poly_trj_5400_7200"
fn = "cm1out_only_upto35.nc"
newncfilepath = os.path.join(dirpath, fn)
print(newncfilepath)

/glade/derecho/scratch/klamb/superdroplets/outsdm_iceball_nowind_rhod_dist_min200_sgs_1024_poly_trj_5400_7200/cm1out_only_upto35.nc


In [266]:
#Opened the netcdf files into a Dataset
nc = xr.open_dataset(newncfilepath)
nc['z'].values

array([ 0.05      ,  0.15      ,  0.25      ,  0.35000002,  0.45000002,
        0.55      ,  0.65000004,  0.75000006,  0.85      ,  0.95000005,
        1.0500001 ,  1.1500001 ,  1.25      ,  1.35      ,  1.45      ,
        1.5500001 ,  1.6500001 ,  1.7500001 ,  1.8500001 ,  1.95      ,
        2.0500002 ,  2.15      ,  2.25      ,  2.3500001 ,  2.45      ,
        2.5500002 ,  2.65      ,  2.7500002 ,  2.8500001 ,  2.95      ,
        3.0500002 ,  3.15      ,  3.2500002 ,  3.3500001 ,  3.45      ,
        3.5500002 ,  3.65      ,  3.7500002 ,  3.8500001 ,  3.9500003 ,
        4.05      ,  4.15      ,  4.25      ,  4.3500004 ,  4.4500003 ,
        4.55      ,  4.65      ,  4.75      ,  4.8500004 ,  4.9500003 ,
        5.05      ,  5.15      ,  5.2500005 ,  5.3500004 ,  5.4500003 ,
        5.55      ,  5.65      ,  5.7500005 ,  5.8500004 ,  5.9500003 ,
        6.05      ,  6.15      ,  6.2500005 ,  6.3500004 ,  6.4500003 ,
        6.55      ,  6.65      ,  6.7500005 ,  6.8500004 ,  6.95

In [267]:
#Selecting certain data variables
selected_vars = ['rh', 'th', 'prs', 'uinterp', 'vinterp', 'winterp', 'out8', 'out9', 'out10', 'out11', 'out12', 'out13', 'out14', 'deactrat', 'xh', 'yh', 'yf', 'z', 'zf']
selected_data = nc[selected_vars]
selected_data

<xarray.Dataset>
Dimensions:   (time: 105, nk: 140, nj: 120, ni: 120, njp1: 121, nkp1: 141)
Coordinates:
  * time      (time) timedelta64[ns] 01:26:00 01:27:00 ... 02:10:00.000488281
Dimensions without coordinates: nk, nj, ni, njp1, nkp1
Data variables: (12/19)
    rh        (time, nk, nj, ni) float32 ...
    th        (time, nk, nj, ni) float32 ...
    prs       (time, nk, nj, ni) float32 ...
    uinterp   (time, nk, nj, ni) float32 ...
    vinterp   (time, nk, nj, ni) float32 ...
    winterp   (time, nk, nj, ni) float32 ...
    ...        ...
    deactrat  (time, nk, nj, ni) float32 ...
    xh        (ni) float32 ...
    yh        (nj) float32 ...
    yf        (njp1) float32 ...
    z         (nk) float32 0.05 0.15 0.25 0.35 0.45 ... 13.65 13.75 13.85 13.95
    zf        (nkp1) float32 ...
Attributes: (12/69)
    cm1 version:  cm1r19.1
    x_units:      km
    x_label:      x
    y_units:      km
    y_label:      y
    z_units:      km
    ...           ...
    cgt1:         1.875
    cgt2:         -1.25
    cgt3:         0.375
    dgt1:         2.0
    dgt2:         -3.0
    dgt3:         1.0

In [259]:
selected_data['xh'].values

array([ 0.05      ,  0.15      ,  0.25      ,  0.35000002,  0.45000002,
        0.55      ,  0.65000004,  0.75000006,  0.85      ,  0.95000005,
        1.0500001 ,  1.1500001 ,  1.25      ,  1.35      ,  1.45      ,
        1.5500001 ,  1.6500001 ,  1.7500001 ,  1.8500001 ,  1.95      ,
        2.0500002 ,  2.15      ,  2.25      ,  2.3500001 ,  2.45      ,
        2.5500002 ,  2.65      ,  2.7500002 ,  2.8500001 ,  2.95      ,
        3.0500002 ,  3.15      ,  3.2500002 ,  3.3500001 ,  3.45      ,
        3.5500002 ,  3.65      ,  3.7500002 ,  3.8500001 ,  3.9500003 ,
        4.05      ,  4.15      ,  4.25      ,  4.3500004 ,  4.4500003 ,
        4.55      ,  4.65      ,  4.75      ,  4.8500004 ,  4.9500003 ,
        5.05      ,  5.15      ,  5.2500005 ,  5.3500004 ,  5.4500003 ,
        5.55      ,  5.65      ,  5.7500005 ,  5.8500004 ,  5.9500003 ,
        6.05      ,  6.15      ,  6.2500005 ,  6.3500004 ,  6.4500003 ,
        6.55      ,  6.65      ,  6.7500005 ,  6.8500004 ,  6.95

In [268]:
#Calculating for the Air Temperature
selected_data['air_temp'] = selected_data['th'] * (selected_data['prs'] / 101325) ** .286
selected_data

<xarray.Dataset>
Dimensions:   (time: 105, nk: 140, nj: 120, ni: 120, njp1: 121, nkp1: 141)
Coordinates:
  * time      (time) timedelta64[ns] 01:26:00 01:27:00 ... 02:10:00.000488281
Dimensions without coordinates: nk, nj, ni, njp1, nkp1
Data variables: (12/20)
    rh        (time, nk, nj, ni) float32 ...
    th        (time, nk, nj, ni) float32 ...
    prs       (time, nk, nj, ni) float32 1.014e+05 1.014e+05 ... 1.484e+04
    uinterp   (time, nk, nj, ni) float32 ...
    vinterp   (time, nk, nj, ni) float32 ...
    winterp   (time, nk, nj, ni) float32 ...
    ...        ...
    xh        (ni) float32 ...
    yh        (nj) float32 ...
    yf        (njp1) float32 ...
    z         (nk) float32 0.05 0.15 0.25 0.35 0.45 ... 13.65 13.75 13.85 13.95
    zf        (nkp1) float32 ...
    air_temp  (time, nk, nj, ni) float64 294.8 294.7 294.7 ... 211.7 211.7 211.7
Attributes: (12/69)
    cm1 version:  cm1r19.1
    x_units:      km
    x_label:      x
    y_units:      km
    y_label:      y
    z_units:      km
    ...           ...
    cgt1:         1.875
    cgt2:         -1.25
    cgt3:         0.375
    dgt1:         2.0
    dgt2:         -3.0
    dgt3:         1.0

In [269]:
# Converts time [ns] in time [seconds]
selected_data['time_seconds'] = nc['time'].values / np.timedelta64(1, 's')
selected_data['time_seconds'] = selected_data['time_seconds'].astype(np.int32)
selected_data

<xarray.Dataset>
Dimensions:       (time: 105, nk: 140, nj: 120, ni: 120, njp1: 121, nkp1: 141,
                   time_seconds: 105)
Coordinates:
  * time          (time) timedelta64[ns] 01:26:00 ... 02:10:00.000488281
  * time_seconds  (time_seconds) int32 5160 5220 5280 5340 ... 7680 7740 7800
Dimensions without coordinates: nk, nj, ni, njp1, nkp1
Data variables: (12/20)
    rh            (time, nk, nj, ni) float32 ...
    th            (time, nk, nj, ni) float32 ...
    prs           (time, nk, nj, ni) float32 1.014e+05 1.014e+05 ... 1.484e+04
    uinterp       (time, nk, nj, ni) float32 ...
    vinterp       (time, nk, nj, ni) float32 ...
    winterp       (time, nk, nj, ni) float32 ...
    ...            ...
    xh            (ni) float32 ...
    yh            (nj) float32 ...
    yf            (njp1) float32 ...
    z             (nk) float32 0.05 0.15 0.25 0.35 ... 13.65 13.75 13.85 13.95
    zf            (nkp1) float32 ...
    air_temp      (time, nk, nj, ni) float64 294.8 294.7 294.7 ... 211.7 211.7
Attributes: (12/69)
    cm1 version:  cm1r19.1
    x_units:      km
    x_label:      x
    y_units:      km
    y_label:      y
    z_units:      km
    ...           ...
    cgt1:         1.875
    cgt2:         -1.25
    cgt3:         0.375
    dgt1:         2.0
    dgt2:         -3.0
    dgt3:         1.0

In [270]:
#Selects only certain times in seconds
selected_data = selected_data.sel(time_seconds = selected_data.time_seconds.isin([5160, 5220]))
selected_data

<xarray.Dataset>
Dimensions:       (time: 105, nk: 140, nj: 120, ni: 120, njp1: 121, nkp1: 141,
                   time_seconds: 4)
Coordinates:
  * time          (time) timedelta64[ns] 01:26:00 ... 02:10:00.000488281
  * time_seconds  (time_seconds) int32 5160 5220 5160 5220
Dimensions without coordinates: nk, nj, ni, njp1, nkp1
Data variables: (12/20)
    rh            (time, nk, nj, ni) float32 ...
    th            (time, nk, nj, ni) float32 ...
    prs           (time, nk, nj, ni) float32 1.014e+05 1.014e+05 ... 1.484e+04
    uinterp       (time, nk, nj, ni) float32 ...
    vinterp       (time, nk, nj, ni) float32 ...
    winterp       (time, nk, nj, ni) float32 ...
    ...            ...
    xh            (ni) float32 ...
    yh            (nj) float32 ...
    yf            (njp1) float32 ...
    z             (nk) float32 0.05 0.15 0.25 0.35 ... 13.65 13.75 13.85 13.95
    zf            (nkp1) float32 ...
    air_temp      (time, nk, nj, ni) float64 294.8 294.7 294.7 ... 211.7 211.7
Attributes: (12/69)
    cm1 version:  cm1r19.1
    x_units:      km
    x_label:      x
    y_units:      km
    y_label:      y
    z_units:      km
    ...           ...
    cgt1:         1.875
    cgt2:         -1.25
    cgt3:         0.375
    dgt1:         2.0
    dgt2:         -3.0
    dgt3:         1.0

In [271]:
# Converts the z position from km to m
selected_data['z'] = selected_data['z'] * 1000
selected_data

<xarray.Dataset>
Dimensions:       (time: 105, nk: 140, nj: 120, ni: 120, njp1: 121, nkp1: 141,
                   time_seconds: 4)
Coordinates:
  * time          (time) timedelta64[ns] 01:26:00 ... 02:10:00.000488281
  * time_seconds  (time_seconds) int32 5160 5220 5160 5220
Dimensions without coordinates: nk, nj, ni, njp1, nkp1
Data variables: (12/20)
    rh            (time, nk, nj, ni) float32 ...
    th            (time, nk, nj, ni) float32 ...
    prs           (time, nk, nj, ni) float32 1.014e+05 1.014e+05 ... 1.484e+04
    uinterp       (time, nk, nj, ni) float32 ...
    vinterp       (time, nk, nj, ni) float32 ...
    winterp       (time, nk, nj, ni) float32 ...
    ...            ...
    xh            (ni) float32 ...
    yh            (nj) float32 ...
    yf            (njp1) float32 ...
    z             (nk) float32 50.0 150.0 250.0 ... 1.385e+04 1.395e+04
    zf            (nkp1) float32 ...
    air_temp      (time, nk, nj, ni) float64 294.8 294.7 294.7 ... 211.7 211.7
Attributes: (12/69)
    cm1 version:  cm1r19.1
    x_units:      km
    x_label:      x
    y_units:      km
    y_label:      y
    z_units:      km
    ...           ...
    cgt1:         1.875
    cgt2:         -1.25
    cgt3:         0.375
    dgt1:         2.0
    dgt2:         -3.0
    dgt3:         1.0

In [272]:
#convert x km to meters
selected_data['xh'] = selected_data['xh'] * 1000
selected_data

<xarray.Dataset>
Dimensions:       (time: 105, nk: 140, nj: 120, ni: 120, njp1: 121, nkp1: 141,
                   time_seconds: 4)
Coordinates:
  * time          (time) timedelta64[ns] 01:26:00 ... 02:10:00.000488281
  * time_seconds  (time_seconds) int32 5160 5220 5160 5220
Dimensions without coordinates: nk, nj, ni, njp1, nkp1
Data variables: (12/20)
    rh            (time, nk, nj, ni) float32 ...
    th            (time, nk, nj, ni) float32 ...
    prs           (time, nk, nj, ni) float32 1.014e+05 1.014e+05 ... 1.484e+04
    uinterp       (time, nk, nj, ni) float32 ...
    vinterp       (time, nk, nj, ni) float32 ...
    winterp       (time, nk, nj, ni) float32 ...
    ...            ...
    xh            (ni) float32 50.0 150.0 250.0 ... 1.185e+04 1.195e+04
    yh            (nj) float32 ...
    yf            (njp1) float32 ...
    z             (nk) float32 50.0 150.0 250.0 ... 1.385e+04 1.395e+04
    zf            (nkp1) float32 ...
    air_temp      (time, nk, nj, ni) float64 294.8 294.7 294.7 ... 211.7 211.7
Attributes: (12/69)
    cm1 version:  cm1r19.1
    x_units:      km
    x_label:      x
    y_units:      km
    y_label:      y
    z_units:      km
    ...           ...
    cgt1:         1.875
    cgt2:         -1.25
    cgt3:         0.375
    dgt1:         2.0
    dgt2:         -3.0
    dgt3:         1.0

In [273]:
#convert y km to meters
selected_data['yh'] = selected_data['yh'] * 1000
selected_data

<xarray.Dataset>
Dimensions:       (time: 105, nk: 140, nj: 120, ni: 120, njp1: 121, nkp1: 141,
                   time_seconds: 4)
Coordinates:
  * time          (time) timedelta64[ns] 01:26:00 ... 02:10:00.000488281
  * time_seconds  (time_seconds) int32 5160 5220 5160 5220
Dimensions without coordinates: nk, nj, ni, njp1, nkp1
Data variables: (12/20)
    rh            (time, nk, nj, ni) float32 ...
    th            (time, nk, nj, ni) float32 ...
    prs           (time, nk, nj, ni) float32 1.014e+05 1.014e+05 ... 1.484e+04
    uinterp       (time, nk, nj, ni) float32 ...
    vinterp       (time, nk, nj, ni) float32 ...
    winterp       (time, nk, nj, ni) float32 ...
    ...            ...
    xh            (ni) float32 50.0 150.0 250.0 ... 1.185e+04 1.195e+04
    yh            (nj) float32 50.0 150.0 250.0 ... 1.185e+04 1.195e+04
    yf            (njp1) float32 ...
    z             (nk) float32 50.0 150.0 250.0 ... 1.385e+04 1.395e+04
    zf            (nkp1) float32 ...
    air_temp      (time, nk, nj, ni) float64 294.8 294.7 294.7 ... 211.7 211.7
Attributes: (12/69)
    cm1 version:  cm1r19.1
    x_units:      km
    x_label:      x
    y_units:      km
    y_label:      y
    z_units:      km
    ...           ...
    cgt1:         1.875
    cgt2:         -1.25
    cgt3:         0.375
    dgt1:         2.0
    dgt2:         -3.0
    dgt3:         1.0

In [290]:
selected_data = selected_data['time_seconds'].sel(5160)
selected_data

TypeError: '_LocIndexer' object is not callable

In [274]:
#Finds the unique times in ascending order
time = selected_data['time_seconds'].values
sorted_time = time

np.unique(np.sort(sorted_time))

array([5160, 5220], dtype=int32)

# Trajectory File

In [205]:
dirpath = '/glade/derecho/scratch/klamb/superdroplets/outsdm_iceball_nowind_rhod_dist_min200_sgs_1024_poly_trj_5400_7200'

In [282]:
# THIS HAS 15 SECONDS
# trajs = load_trajectories(dirpath,num_timesteps=10)
timestamps = get_timestamps(dirpath)

# load every 4 timestamps (every 1 minute)
coarse_timestamps = timestamps[0::4]

# only look at 10 superdroplets 
Ns = 10
unique_superdroplets = get_unique_SDs(dirpath, coarse_timestamps[0])
first_Ns = unique_superdroplets[0:Ns]

print(coarse_timestamps)
print(first_Ns)
trajs = load_trajectories(dirpath, times=coarse_timestamps,
                         num_timesteps = 3, 
                         Ns_array = first_Ns)

[5100 5160 5220 5280 5340 5400 5460 5520 5580 5640 5700 5760 5820 5880
 5940 6000 6060 6120 6180 6240 6300 6360 6420 6480 6540 6600 6660 6720
 6780 6840 6900 6960 7020 7080 7140 7200]
[ 409  551  781  872 1422 1464 1934 3033 3531 3883]
Loading trajectories for time 5100 
Loading trajectories for time 5160 
Loading trajectories for time 5220 


In [284]:
trajs = trajs[trajs["time"] == 5160]
trajs

,x[m],y[m],z[m],vz[m],radius(droplet)[m],mass_of_aerosol_in_droplet/ice(1:01)[g],radius_eq(ice)[m],radius_pol(ice)[m],density(droplet/ice)[kg/m3],rhod [kg/m3],multiplicity[-],status[-],index,rime_mass[kg],num_of_monomers[-],time
rk_deact,,,,,,,,,,,,,,,,
409,260.60377,368.385200,9213.5906,0.0,0.0,4.152454e-13,0.000011,0.000011,200.59303,108.18,15177056256,ICE,409,0.0,1,5160
551,407.87111,210.440490,9741.4169,0.0,0.0,1.412397e-14,0.000027,0.000027,200.00253,99.55,74482008064,ICE,551,0.0,1,5160
781,411.10460,434.788160,9654.6669,0.0,0.0,7.027485e-14,0.000019,0.000019,357.11953,358.64,58047827968,ICE,781,0.0,1,5160
872,233.15415,383.453870,9512.1130,0.0,0.0,1.622385e-13,0.000045,0.000045,289.06606,289.55,35479605248,ICE,872,0.0,1,5160
1422,479.72596,38.228922,9674.1355,0.0,0.0,2.185584e-13,0.000019,0.000019,362.34346,410.45,28040101888,ICE,1422,0.0,1,5160
1464,321.83001,372.833130,9657.9312,0.0,0.0,1.477063e-15,0.000007,0.000007,652.29955,712.73,50299813888,ICE,1464,0.0,1,5160
1934,341.39308,112.406710,9210.4875,0.0,0.0,2.507824e-13,0.000070,0.000070,203.15980,203.18,24888346624,ICE,1934,0.0,1,5160
3033,471.56190,469.990120,9602.1861,0.0,0.0,1.013003e-14,0.000012,0.000012,304.28089,315.45,69965635584,ICE,3033,0.0,1,5160
3531,419.31608,167.016510,9663.3729,0.0,0.0,5.652341e-14,0.000047,0.000047,200.00119,151.36,63349731328,ICE,3531,0.0,1,5160


In [285]:
# Z position
superdroplets_list = [409, 551, 781, 872, 1422, 1464, 1934, 3033, 3531, 3883]

selected_data = selected_data.sel(time_seconds = 5160)
z_midpoints = selected_data['z'].values
min_array = []
final_j = -1
z_index_array = []

for i in range(len(superdroplets_list)):
    for j in range(120):
        current_min = abs(x_midpoints[j] - trajs.loc[superdroplets_list[i],'z[m]'])
        if current_min < 50.0:
            final_j = j
            z_index_array.append(final_j)
            min_array.append(current_min)

print(z_index_array)

[92, 97, 96, 95, 96, 96, 92, 96, 96, 91]


In [225]:
#X position
superdroplets_list = [409, 551, 781, 872, 1422, 1464, 1934, 3033, 3531, 3883]
time_list = [5160, 5220]

selected_data = selected_data.sel(time_seconds = 5160)
x_midpoints = selected_data['xh'].values
min_array = []
final_j = -1
x_index_array = []

for i in range(len(superdroplets_list)):
    for j in range(120):
        current_min = abs(x_midpoints[j] - trajs.loc[superdroplets_list[i],'x[m]'])
        if current_min < 50.0:
            final_j = j
            x_index_array.append(final_j)
            min_array.append(current_min)

print(x_index_array)

[2, 4, 4, 2, 4, 3, 3, 4, 4, 3]


In [286]:
#Y position
superdroplets_list = [409, 551, 781, 872, 1422, 1464, 1934, 3033, 3531, 3883]
time_list = [5160, 5220]

selected_data = selected_data.sel(time_seconds = 5160)
y_midpoints = selected_data['yh'].values
min_array = []
final_j = -1
y_index_array = []

for i in range(len(superdroplets_list)):
    for j in range(120):
        current_min = abs(y_midpoints[j] - trajs.loc[superdroplets_list[i],'y[m]'])
        if current_min < 50.0:
            final_j = j
            y_index_array.append(final_j)
            min_array.append(current_min)

print(y_index_array)

[3, 2, 4, 3, 0, 3, 1, 4, 1, 4]


In [287]:
#Combine Indices and Append them to the Trajectory DataFrame
coordinates = []
for i, j, k in zip(x_index_array, y_index_array, z_index_array):
    tuple = (i, j, k)
    coordinates.append(tuple)
coordinates

[(2, 3, 92),
 (4, 2, 97),
 (4, 4, 96),
 (2, 3, 95),
 (4, 0, 96),
 (3, 3, 96),
 (3, 1, 92),
 (4, 4, 96),
 (4, 1, 96),
 (3, 4, 91)]

In [288]:
trajs['ijk coordinates'] = coordinates
trajs

/glade/derecho/scratch/ashleyn/tmp/ipykernel_72103/49819130.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trajs['ijk coordinates'] = coordinates


,x[m],y[m],z[m],vz[m],radius(droplet)[m],mass_of_aerosol_in_droplet/ice(1:01)[g],radius_eq(ice)[m],radius_pol(ice)[m],density(droplet/ice)[kg/m3],rhod [kg/m3],multiplicity[-],status[-],index,rime_mass[kg],num_of_monomers[-],time,ijk coordinates
rk_deact,,,,,,,,,,,,,,,,,
409,260.60377,368.385200,9213.5906,0.0,0.0,4.152454e-13,0.000011,0.000011,200.59303,108.18,15177056256,ICE,409,0.0,1,5160,"(2, 3, 92)"
551,407.87111,210.440490,9741.4169,0.0,0.0,1.412397e-14,0.000027,0.000027,200.00253,99.55,74482008064,ICE,551,0.0,1,5160,"(4, 2, 97)"
781,411.10460,434.788160,9654.6669,0.0,0.0,7.027485e-14,0.000019,0.000019,357.11953,358.64,58047827968,ICE,781,0.0,1,5160,"(4, 4, 96)"
872,233.15415,383.453870,9512.1130,0.0,0.0,1.622385e-13,0.000045,0.000045,289.06606,289.55,35479605248,ICE,872,0.0,1,5160,"(2, 3, 95)"
1422,479.72596,38.228922,9674.1355,0.0,0.0,2.185584e-13,0.000019,0.000019,362.34346,410.45,28040101888,ICE,1422,0.0,1,5160,"(4, 0, 96)"
1464,321.83001,372.833130,9657.9312,0.0,0.0,1.477063e-15,0.000007,0.000007,652.29955,712.73,50299813888,ICE,1464,0.0,1,5160,"(3, 3, 96)"
1934,341.39308,112.406710,9210.4875,0.0,0.0,2.507824e-13,0.000070,0.000070,203.15980,203.18,24888346624,ICE,1934,0.0,1,5160,"(3, 1, 92)"
3033,471.56190,469.990120,9602.1861,0.0,0.0,1.013003e-14,0.000012,0.000012,304.28089,315.45,69965635584,ICE,3033,0.0,1,5160,"(4, 4, 96)"
3531,419.31608,167.016510,9663.3729,0.0,0.0,5.652341e-14,0.000047,0.000047,200.00119,151.36,63349731328,ICE,3531,0.0,1,5160,"(4, 1, 96)"


In [ ]:
# Add pressure
